<a href="https://colab.research.google.com/github/yolandaeka/DataMining_2024/blob/main/outliers2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Loading and Decribing Example Data

In [5]:
#improt dependencies
import pandas as pd
import numpy as np
import plotly.express as px

#load data into a dataframe

df = pd.read_csv('uber.csv')

#check the first 5 rows
df.head()

#drop unnecessary columns
df = df.drop(columns = (['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']))


In [6]:
df.head()

Unnamed: 0                            key  fare_amount  \
0    24238194    2015-05-07 19:52:06.0000003          7.5   
1    27835199    2009-07-17 20:04:56.0000002          7.7   
2    44984355   2009-08-24 21:45:00.00000061         12.9   
3    25894730    2009-06-26 08:22:21.0000001          5.3   
4    17610152  2014-08-28 17:47:00.000000188         16.0   

           pickup_datetime  passenger_count  
0  2015-05-07 19:52:06 UTC                1  
1  2009-07-17 20:04:56 UTC                1  
2  2009-08-24 21:45:00 UTC                1  
3  2009-06-26 08:22:21 UTC                3  
4  2014-08-28 17:47:00 UTC                5

Using pandas describe() to find outliers

In [8]:
df.describe()[['fare_amount', 'passenger_count']]

fare_amount  passenger_count
count  200000.000000    200000.000000
mean       11.359955         1.684535
std         9.901776         1.385997
min       -52.000000         0.000000
25%         6.000000         1.000000
50%         8.500000         1.000000
75%        12.500000         2.000000
max       499.000000       208.000000

Find outliers and view the data distribution using a histogram

In [9]:
#create a histogram
fig = px.histogram(df, x = 'fare_amount')
fig.show()

Find outliers in data using a box plot

In [10]:
#create a box plot

fig = px.box(df, y = "fare_amount")
fig.show()

Find multivariate outliers using a scatter plot

In [11]:
fig = px.scatter(x = df['passenger_count'], y = df['fare_amount'])
fig.show()

Finding outliers using statistical methods

In [12]:
#create a function to find outliers using IQR

def find_outliers_IQR(df):

   q1=df.quantile(0.25)

   q3=df.quantile(0.75)

   IQR=q3-q1

   outliers = df[((df<(q1-1.5*IQR)) | (df>(q3+1.5*IQR)))]

   return outliers

In [15]:
outliers = find_outliers_IQR(df['fare_amount'])

print('number of outliers: '+ str(len(outliers)))

print('max outlier value: '+ str(outliers.max()))

print('min outlier value: '+ str(outliers.min()))

outliers

number of outliers: 17167
max outlier value: 499.0
min outlier value: -52.0


6         24.50
30        25.70
34        39.50
39        29.00
48        56.80
          ...  
199976    49.70
199977    43.50
199982    57.33
199985    24.00
199997    30.90
Name: fare_amount, Length: 17167, dtype: float64

In [17]:
outliers = find_outliers_IQR(df[['passenger_count','fare_amount']])

outliers

passenger_count  fare_amount
0                   NaN          NaN
1                   NaN          NaN
2                   NaN          NaN
3                   NaN          NaN
4                   5.0          NaN
...                 ...          ...
199995              NaN          NaN
199996              NaN          NaN
199997              NaN         30.9
199998              NaN          NaN
199999              NaN          NaN

[200000 rows x 2 columns]

Working with outliers using statistical methods

Drop the outliers

In [22]:
def drop_outliers_IQR(df):

   q1=df.quantile(0.25)

   q3=df.quantile(0.75)

   IQR=q3-q1

   not_outliers = df[~((df<(q1-1.5*IQR)) | (df>(q3+1.5*IQR)))]

   outliers_dropped = outliers.dropna().reset_index()

   return outliers_dropped

Cap the outliers

In [25]:
upper_limit = df['fare_amount'].mean() + 3*df['fare_amount'].std()

print(upper_limit)

lower_limit = df['fare_amount'].mean() - 3*df['fare_amount'].std()

print(lower_limit)

41.0652839252097
-18.345373425209697


In [27]:
df['fare_amount'] = np.where(df['fare_amount'] > upper_limit,

   upper_limit,

   np.where(

       df['fare_amount'] < lower_limit,

       lower_limit,

       df['fare_amount']

   )

)

In [29]:
df.describe()[['fare_amount']]

fare_amount
count  200000.000000
mean       11.008988
std         8.088084
min       -18.345373
25%         6.000000
50%         8.500000
75%        12.500000
max        41.065284

Replace outliers using imputation as if they were missing values

In [30]:
def impute_outliers_IQR(df):

   q1=df.quantile(0.25)

   q3=df.quantile(0.75)

   IQR=q3-q1

   upper = df[~(df>(q3+1.5*IQR))].max()

   lower = df[~(df<(q1-1.5*IQR))].min()

   df = np.where(df > upper,

       df.mean(),

       np.where(

           df < lower,

           df.mean(),

           df

           )

       )

   return df

In [31]:
df['fare_amount'] = impute_outliers_IQR(df['fare_amount'])

df.describe()['fare_amount']

count    200000.000000
mean          9.117679
std           4.003623
min          -3.500000
25%           6.000000
50%           8.500000
75%          11.008988
max          22.200000
Name: fare_amount, dtype: float64